In [1]:
import pymongo
from pymongo import MongoClient,ReturnDocument
from bson import ObjectId
from transformers import pipeline


In [2]:
nlp = pipeline("ner")

In [34]:
client = MongoClient('localhost', 27017)
db = client.db_world_bank
reports_coll = db["reports"]

coll_knowledge_notes = reports_coll.find({'collection':'Knowledge Notes','full_text': {'$exists':True}})                                         
print(coll_knowledge_notes.count())
for elem in coll_knowledge_notes[1:2]:
    txt=elem['full_text']
    


352


<ipython-input-34-2f4204e7de92>:6: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  print(coll_knowledge_notes.count())


In [32]:
print(elem)

{'_id': ObjectId('5f770a039d1d71848dfe8813'), 'title': 'Social Protection and Jobs Responses to COVID-19 : A Real-Time Review of Country Measures', 'abstract': 'Some key finds from this "living paper" include : As of April 23, 2020, a total of 151 countries (18 more since last week) have planned, introduced or adapted 684 social protection measures in response to COVID-19 (Coronavirus). This is a ten-fold increase in measures since the first edition of this living paper (March 20). New countries include Angola, Anguilla, Antigua and Barbuda, Aruba, Azerbaijan, Bahamas, Belarus, Bermuda, Brunei, Chad, Grenada, Libya, Montserrat, Nigeria, Saint Vincent and the Grenadines, Seychelles, St Maarten, and UAE. Social assistance transfers are the most widely used class of interventions (60 percent of global responses, or 412 measures). These are complemented by significant action in social insurance and labor market-related measures (supply-side measures). Among safety nets, cash transfer progr

In [35]:
interesting_entities = []
text_length = len(txt)
for start_range in range(0,text_length,10000):
    entities = nlp(txt[start_range:start_range+10000].replace('\n',''))
    temp_lst = [entity for entity in entities if entity["entity"]=="I-LOC"]
    print(temp_lst)
    interesting_entities += temp_lst

[]
[]
[{'word': 'South', 'score': 0.9821796417236328, 'entity': 'I-LOC', 'index': 68}, {'word': 'V', 'score': 0.9948087930679321, 'entity': 'I-LOC', 'index': 72}, {'word': '##en', 'score': 0.9654321074485779, 'entity': 'I-LOC', 'index': 73}, {'word': 'e', 'score': 0.9858993291854858, 'entity': 'I-LOC', 'index': 75}, {'word': '##zu', 'score': 0.9724932312965393, 'entity': 'I-LOC', 'index': 76}, {'word': '##ela', 'score': 0.9964817762374878, 'entity': 'I-LOC', 'index': 77}, {'word': 'Colombia', 'score': 0.9997696280479431, 'entity': 'I-LOC', 'index': 79}, {'word': 'Nepal', 'score': 0.9990834593772888, 'entity': 'I-LOC', 'index': 81}, {'word': 'India', 'score': 0.9996359348297119, 'entity': 'I-LOC', 'index': 83}, {'word': 'Zimbabwe', 'score': 0.9992150664329529, 'entity': 'I-LOC', 'index': 85}, {'word': 'South', 'score': 0.9997557401657104, 'entity': 'I-LOC', 'index': 87}, {'word': 'Africa', 'score': 0.9996945858001709, 'entity': 'I-LOC', 'index': 88}, {'word': 'Myanmar', 'score': 0.99965

[{'word': 'Africa', 'score': 0.939723551273346, 'entity': 'I-LOC', 'index': 132}, {'word': 'Africa', 'score': 0.9867938160896301, 'entity': 'I-LOC', 'index': 155}, {'word': 'Washington', 'score': 0.981979250907898, 'entity': 'I-LOC', 'index': 163}, {'word': 'DC', 'score': 0.9967349767684937, 'entity': 'I-LOC', 'index': 165}, {'word': 'Washington', 'score': 0.9930219054222107, 'entity': 'I-LOC', 'index': 275}, {'word': 'D', 'score': 0.9971830248832703, 'entity': 'I-LOC', 'index': 276}, {'word': 'C', 'score': 0.9988296031951904, 'entity': 'I-LOC', 'index': 278}, {'word': 'Cambodia', 'score': 0.9984305500984192, 'entity': 'I-LOC', 'index': 368}, {'word': 'Democratic', 'score': 0.9984644651412964, 'entity': 'I-LOC', 'index': 379}, {'word': 'Republic', 'score': 0.9988217949867249, 'entity': 'I-LOC', 'index': 380}, {'word': 'Myanmar', 'score': 0.9996461272239685, 'entity': 'I-LOC', 'index': 383}, {'word': 'Thailand', 'score': 0.8336246609687805, 'entity': 'I-LOC', 'index': 388}]
[{'word': 'U

In [31]:
len(interesting_entities)

339

In [23]:
interesting_entities = [entity for entity in entities if entity["entity"]=="I-LOC"]
interesting_entities

[{'word': 'Philippines',
  'score': 0.999841034412384,
  'entity': 'I-LOC',
  'index': 118},
 {'word': 'El', 'score': 0.9995218515396118, 'entity': 'I-LOC', 'index': 120},
 {'word': 'Salvador',
  'score': 0.999811053276062,
  'entity': 'I-LOC',
  'index': 121},
 {'word': '##A', 'score': 0.9263363480567932, 'entity': 'I-LOC', 'index': 127},
 {'word': '##rgent',
  'score': 0.9982383847236633,
  'entity': 'I-LOC',
  'index': 128},
 {'word': '##ina',
  'score': 0.9904986619949341,
  'entity': 'I-LOC',
  'index': 129},
 {'word': 'Colombia',
  'score': 0.9998371005058289,
  'entity': 'I-LOC',
  'index': 131},
 {'word': 'Morocco',
  'score': 0.9998611211776733,
  'entity': 'I-LOC',
  'index': 143},
 {'word': 'Philippines',
  'score': 0.9995419383049011,
  'entity': 'I-LOC',
  'index': 203},
 {'word': 'E', 'score': 0.8840664029121399, 'entity': 'I-LOC', 'index': 211},
 {'word': 'Salvador',
  'score': 0.9931319355964661,
  'entity': 'I-LOC',
  'index': 213},
 {'word': 'Chile',
  'score': 0.9996

In [19]:
entities

[{'word': 'n', 'score': 0.6234421730041504, 'entity': 'I-PER', 'index': 1},
 {'word': 'Plaza', 'score': 0.9994154572486877, 'entity': 'I-PER', 'index': 3},
 {'word': 'Victoria',
  'score': 0.9997586011886597,
  'entity': 'I-PER',
  'index': 5},
 {'word': 'St', 'score': 0.9997323751449585, 'entity': 'I-PER', 'index': 6},
 {'word': '##rok', 'score': 0.9977715611457825, 'entity': 'I-PER', 'index': 7},
 {'word': '##ova', 'score': 0.9974404573440552, 'entity': 'I-PER', 'index': 8},
 {'word': 'Anna', 'score': 0.9997384548187256, 'entity': 'I-PER', 'index': 10},
 {'word': 'Su', 'score': 0.999614953994751, 'entity': 'I-PER', 'index': 11},
 {'word': '##kh', 'score': 0.996711015701294, 'entity': 'I-PER', 'index': 12},
 {'word': '##ova',
  'score': 0.9961882829666138,
  'entity': 'I-PER',
  'index': 13},
 {'word': 'Chang',
  'score': 0.9989685416221619,
  'entity': 'I-PER',
  'index': 15},
 {'word': '##g', 'score': 0.9986360669136047, 'entity': 'I-PER', 'index': 16},
 {'word': '##qing',
  'score'

In [12]:
interesting_entities

[]